# Data Preprocessing

---





In [2]:
# Run this cell to authenticate yourself to BigQuery
from google.colab import auth
auth.authenticate_user() # make sure to "Select All" when authenticating
project_id = "cs145-project1-474600"

# Run this cell to authenticate yourself to BigQuery
from google.colab import auth
auth.authenticate_user() # make sure to "Select All" when authenticating
project_id = "cs145-project1-474600"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Section 1: Load + Convert to DataFrame from BigQuery Dataset

In [ ]:
%%bigquery --project $project_id

SELECT
  d.subject_id,
  d.hadm_id,
  a.admittime AS timestamp,
  d.icd_code,
  icd.long_title AS diagnosis
FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd` AS d
JOIN `physionet-data.mimiciv_3_1_hosp.admissions` AS a
    ON d.hadm_id = a.hadm_id
LEFT JOIN `physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses` AS icd
    ON d.icd_code = icd.icd_code
    AND d.icd_version = icd.icd_version
ORDER BY d.subject_id, a.admittime
LIMIT 50



ERROR:
 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/cs145-project1-474600/jobs?prettyPrint=false: Access Denied: Project cs145-project1-474600: User does not have bigquery.jobs.create permission in project cs145-project1-474600.

Location: None
Job ID: a3bad7c9-e2ce-47a2-9d6c-5546e2c086c0



In [ ]:
from google.cloud import bigquery
import pandas as pd
import os

# Pull data to dataframe
from google.colab import auth
auth.authenticate_user()

client = bigquery.Client(project=project_id)

query = """
SELECT
  d.subject_id,
  d.hadm_id,
  a.admittime AS timestamp,
  d.icd_code,
  icd.long_title AS diagnosis
FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd` AS d
JOIN `physionet-data.mimiciv_3_1_hosp.admissions` AS a
    ON d.hadm_id = a.hadm_id
LEFT JOIN `physionet-data.mimiciv_3_1_hosp.d_icd_diagnoses` AS icd
    ON d.icd_code = icd.icd_code
    AND d.icd_version = icd.icd_version
ORDER BY d.subject_id, a.admittime
"""

print("Running BigQuery query…")
df = client.query(query).to_dataframe()
print("Rows loaded:", len(df))


Running BigQuery query…


Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/cs145-project1-474600/jobs?prettyPrint=false: Access Denied: Project cs145-project1-474600: User does not have bigquery.jobs.create permission in project cs145-project1-474600.

Location: None
Job ID: 968a6b2d-8c54-4328-a199-0a55b93861b9


##### Section 2: Build + Save Temporal Disease Graph

In [ ]:
df = df.dropna(subset=["timestamp", "icd_code"])
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values(["subject_id", "timestamp"])

edges = []

for pid, group in df.groupby("subject_id"):
    codes = list(group["icd_code"])
    times = list(group["timestamp"])
    for i in range(len(codes) - 1):
        edges.append((codes[i], codes[i+1], times[i+1]))

edges_df = pd.DataFrame(edges, columns=["src", "dst", "timestamp"])
nodes_df = pd.DataFrame(df["icd_code"].unique(), columns=["disease_code"])


In [ ]:
output_dir = "/content/drive/MyDrive/CS224W/mimic_preprocessed"
os.makedirs(output_dir, exist_ok=True)

edges_df.to_csv(f"{output_dir}/disease_edges.csv", index=False)
nodes_df.to_csv(f"{output_dir}/disease_nodes.csv", index=False)


# Dataset Statistics


---



In [9]:
# Retrieve graph nodes and edges
import pandas as pd
from google.colab import drive
import os
output_dir = "drive/MyDrive/CS224W/mimic_preprocessed"
# os.makedirs(output_dir, exist_ok=True)
drive.mount('/content/drive/')


disease_nodes_df = pd.read_csv(os.path.join(output_dir, "disease_nodes.csv"))
disease_edges_df = pd.read_csv(os.path.join(output_dir, "disease_edges.csv"))

Mounted at /content/drive/


In [11]:
import matplotlib.pyplot as plt
import networkx as nx

edge_counts = disease_edges_df.groupby(['src', 'dst']).size().reset_index(name='weight')
top_edges = edge_counts.sort_values('weight', ascending=False).head(30)

print("Top 10 most frequent disease transitions (Source, Destination, Count):")
print(top_edges.head(10).to_string(index=False))


Top 10 most frequent disease transitions (Source, Destination, Count):
   src    dst  weight
  4019   2724    4046
  2724   4019    3902
  E785    I10    3544
   I10   E785    3448
  4019  25000    2726
 25000   4019    2632
 53081   4019    2255
  4019  53081    2207
Z87891   E785    2062
   I10 Z87891    1935
